In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('C:\\Users\\Hp\\Downloads\\covid_toy.csv')

In [3]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test , y_train, y_test = train_test_split(df.drop(columns = ['has_covid']),df['has_covid'], test_size=0.2)

In [5]:
x_train

,age,gender,fever,cough,city
20,12,Male,98.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
72,83,Female,101.0,Mild,Kolkata
37,55,Male,100.0,Mild,Kolkata
50,19,Male,101.0,Mild,Delhi
...,...,...,...,...,...
54,60,Female,99.0,Mild,Mumbai
77,8,Female,101.0,Mild,Kolkata
91,38,Male,NaN,Mild,Delhi
25,23,Male,NaN,Mild,Mumbai


In [6]:
#simple imputer to fill NaN values
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

In [7]:
x_test_fever = si.fit_transform(x_test[['fever']])

In [8]:
#ordinal encoding on cough
oe = OrdinalEncoder(categories = [['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

In [9]:
x_test_cough = oe.fit_transform(x_test[['cough']])

In [10]:
x_train_cough.shape

(80, 1)

In [11]:
#One hot encoding on gender and city
ohe = OneHotEncoder( drop = 'first', sparse = False)

In [12]:
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])

x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
x_train_gender_city.shape

(80, 4)

In [14]:
#extracting age

x_train_age = x_train.drop(columns = ['gender', 'fever','cough', 'city']).values

In [15]:
x_test_age = x_test.drop(columns = ['gender', 'fever','cough', 'city']).values

In [16]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough),axis = 1)

In [17]:
x_train_transformed.shape

(80, 7)

In [18]:
from sklearn.compose import ColumnTransformer
# to apply all three appporaches ek sath instead of using 
#sare methods alag alag as done above
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]),['cough']),
    ('tnf3', OneHotEncoder(sparse = False, drop='first'),['gender','city'])
],remainder = 'passthrough')

In [19]:
transformer.fit_transform(x_train).shape

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [20]:
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split



In [21]:
x_train, x_test , y_train, y_test = train_test_split(df.drop(columns = ['has_covid']),df['has_covid'], test_size=0.2)

In [22]:
categorical_features = ['gender', 'city']
numeric_features = ['age', 'fever']

In [23]:
# create transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [24]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [25]:
#combine transformer
preprocessor = ColumnTransformer(
transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

#Create thhe pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegression())])

#train the model
clf.fit(x_train, y_train)

#Evaluate the model

y_pred = clf.predict(x_test)

In [26]:
y_pred

array(['No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No',
       'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No'],
      dtype=object)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
acc = accuracy_score(y_test,y_pred)

In [29]:
acc

0.6